# Imports

In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["KAGGLE_USERNAME"] = "leonardomarcellino"
os.environ["KAGGLE_KEY"] = "97732c941f9704e4cb24c9c246cd7050"

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatibl

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # Or "tensorflow" or "torch".
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.9"

In [ ]:
import keras
import keras_nlp

# Load dataset

In [ ]:
!pip install jsonlines

In [1]:
import jsonlines

ModuleNotFoundError: No module named 'jsonlines'

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

Untuk remove downloaded dataset stlh diubah ke JSONL

In [ ]:
import shutil
import glob

## dataset ruslanmv/ai-medical-chatbot

In [ ]:
dataset = load_dataset("ruslanmv/ai-medical-chatbot")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 256916
    })
})


Extract and save dataset to JSONL format

In [ ]:
# Extract columns from train set
train_data = dataset['train'].shuffle(seed=42)
descriptions = train_data['Description']
patients = train_data['Patient']
doctors = train_data['Doctor']

In [ ]:
# Save to JSONL
with jsonlines.open('ai_medical_chatbot_train.jsonl', mode='w') as writer:
    for description, patient, doctor in zip(descriptions, patients, doctors):
        writer.write({
            'Description': description,
            'Patient': patient,
            'Doctor': doctor
        })

Delete downloaded dataset

In [ ]:
# Delete all potential cached dataset directories
cache_dirs = set()

# Adding dataset specific cache directories
for split in dataset.keys():
    for cache_file in dataset[split].cache_files:
        cache_dirs.add(os.path.dirname(cache_file['filename']))

# Add common dataset cache directories (if any)
cache_dirs.add(os.path.expanduser("~/.cache/huggingface/datasets"))
cache_dirs.add("/root/.cache/huggingface/datasets")

# Also check common directories in colab environment
cache_dirs.update(glob.glob('/root/.cache/huggingface/datasets/**/', recursive=True))

for cache_dir in cache_dirs:
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)
        print(f"Removed cache directory: {cache_dir}")

Removed cache directory: /root/.cache/huggingface/datasets/ruslanmv___ai-medical-dataset/default/
Removed cache directory: /root/.cache/huggingface/datasets/ruslanmv___ai-medical-chatbot/default/0.0.0/138c99336a3afce0df88ffe6fd67bd231df25d36/
Removed cache directory: /root/.cache/huggingface/datasets/ruslanmv___ai-medical-chatbot/default/
Removed cache directory: /root/.cache/huggingface/datasets/


Check the data

In [ ]:
train_data[0]

{'Description': 'Appendix cancer, adino carcinoma, removed partial intestine. Suggestion?',
 'Patient': 'last year my wife was went through a surgery for appendix cancer, that appendix was removed , that appendix slice tested in lab and found so called adino carcinoma in apedix,  after that doctor decided to operate again and remove her partial intestine, there was no sign of cancer in any test other than the biopsy of appendix, however  after one moth of hospitalization came back to home, 6 month of follow up check up no bad sign, now almost one year of surgery puss mark notice at steches near belly button .Please advice this is not a sign of any cancer',
 'Doctor': 'Hi and welcome to HCM. First, you dont have to worry. This cant be tumour relaps because this is lesion in abdominall wall,obviously some local infection or wound abscess.This is often seen after laparotomy. Appendix cancers are rare but in most cases surgery is enough for complete treatment and recovery. If tehre were no

In [ ]:
# Buka file JSONL
with jsonlines.open('ai_medical_chatbot_train.jsonl', mode='r') as reader:
    # Baca satu baris pertama
    first_line = reader.read()

# Cetak baris pertama
print(first_line)

{'Description': 'Appendix cancer, adino carcinoma, removed partial intestine. Suggestion?', 'Patient': 'last year my wife was went through a surgery for appendix cancer, that appendix was removed , that appendix slice tested in lab and found so called adino carcinoma in apedix,  after that doctor decided to operate again and remove her partial intestine, there was no sign of cancer in any test other than the biopsy of appendix, however  after one moth of hospitalization came back to home, 6 month of follow up check up no bad sign, now almost one year of surgery puss mark notice at steches near belly button .Please advice this is not a sign of any cancer', 'Doctor': 'Hi and welcome to HCM. First, you dont have to worry. This cant be tumour relaps because this is lesion in abdominall wall,obviously some local infection or wound abscess.This is often seen after laparotomy. Appendix cancers are rare but in most cases surgery is enough for complete treatment and recovery. If tehre were no f

In [ ]:
first_line

{'Description': 'Q. What does abutment of the nerve root mean?',
 'Patient': 'Hi doctor,I am just wondering what is abutting and abutment of the nerve root means in a back issue. Please explain. What treatment is required for\xa0annular bulging and tear?',
 'Doctor': 'Hi. I have gone through your query with diligence and would like you to know that I am here to help you. For further information consult a neurologist online -->'}

Save JSONL format dataset to empty list for training

In [ ]:
import jsonlines
import json
# Membuat list kosong untuk menyimpan data
data = []

In [ ]:
# Buka file JSONL
with jsonlines.open('ai_medical_chatbot_train.jsonl', mode='r') as reader:
    # Membaca setiap baris dari file
    for features in reader:
        # Format teks instruksi dan tanggapan
        template = "Instruction:\n{Patient}\n\nResponse:\n{Doctor}"
        data.append(template.format(**features))

# Cetak data (10 contoh pertama)
print(data[:10])

['Instruction:\nlast year my wife was went through a surgery for appendix cancer, that appendix was removed , that appendix slice tested in lab and found so called adino carcinoma in apedix,  after that doctor decided to operate again and remove her partial intestine, there was no sign of cancer in any test other than the biopsy of appendix, however  after one moth of hospitalization came back to home, 6 month of follow up check up no bad sign, now almost one year of surgery puss mark notice at steches near belly button .Please advice this is not a sign of any cancer\n\nResponse:\nHi and welcome to HCM. First, you dont have to worry. This cant be tumour relaps because this is lesion in abdominall wall,obviously some local infection or wound abscess.This is often seen after laparotomy. Appendix cancers are rare but in most cases surgery is enough for complete treatment and recovery. If tehre were no found metastasis or extended disease during the surgery, you dont have to expect relaps.

In [ ]:
len(data)

256916

In [ ]:
print(data[0])

Instruction:
last year my wife was went through a surgery for appendix cancer, that appendix was removed , that appendix slice tested in lab and found so called adino carcinoma in apedix,  after that doctor decided to operate again and remove her partial intestine, there was no sign of cancer in any test other than the biopsy of appendix, however  after one moth of hospitalization came back to home, 6 month of follow up check up no bad sign, now almost one year of surgery puss mark notice at steches near belly button .Please advice this is not a sign of any cancer

Response:
Hi and welcome to HCM. First, you dont have to worry. This cant be tumour relaps because this is lesion in abdominall wall,obviously some local infection or wound abscess.This is often seen after laparotomy. Appendix cancers are rare but in most cases surgery is enough for complete treatment and recovery. If tehre were no found metastasis or extended disease during the surgery, you dont have to expect relaps. If th

In [ ]:
#tdk dipakai
'''
import json
data = []
with open("ai_medical_chatbot_train.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["Description"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{Patient}\n\nResponse:\n{Doctor}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]
'''

## dataset ruslanmv/ai-medical-dataset

In [ ]:
dataset = load_dataset("ruslanmv/ai-medical-dataset")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21210000 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'context'],
        num_rows: 21210000
    })
})

In [ ]:
train_data = dataset['train'].shuffle(seed=42)
train_data = train_data[:500]
question = train_data['question']
context = train_data['context']

In [ ]:
train_data

{'question': ['',
  'What is the use of triage tags widely advocated as a tool to improve the management',
  '',
  'What is the main reason for melancholic depression?',
  '',
  '',
  '',
  'What is the EXERCISE CAPACITY AND DYSPNEA IN O',
  'What is the name of the mesh that was soaked in saline?',
  'What is the combination chemotherapy of intraperitoneal carboplatin and intravenous ',
  'What is the name of the SPECIAL WARNING(S)?',
  'Age less than 18 years old',
  '',
  '',
  'Uncontrolled ischemic heart disease or uncontrolled congestive heart failure - Uncontrolled',
  '',
  'What is the DNA encoded for?',
  'How many depressive inpatients were examined after admission?',
  'What is the name of the patient in emergency situations?',
  'What is the primary endpoint of ADs at follow-up?',
  'What is the decision to surgical management of incomplete miscarriage by the health care team?',
  'What is the name of the gene family expressed in rat liver?',
  'What is the prevalence of p

Extract and save dataset to JSONL format

In [ ]:
# Save to JSONL
with jsonlines.open('ai_medical_dataset_train.jsonl', mode='w') as writer:
    for description, patient, doctor in zip(descriptions, patients, doctors):
        writer.write({
            'question': question,
            'context': context
        })

In [ ]:
# Delete all potential cached dataset directories
cache_dirs = set()

# Adding dataset specific cache directories
for split in dataset.keys():
    for cache_file in dataset[split].cache_files:
        cache_dirs.add(os.path.dirname(cache_file['filename']))

# Add common dataset cache directories (if any)
cache_dirs.add(os.path.expanduser("~/.cache/huggingface/datasets"))
cache_dirs.add("/root/.cache/huggingface/datasets")

# Also check common directories in colab environment
cache_dirs.update(glob.glob('/root/.cache/huggingface/datasets/**/', recursive=True))

for cache_dir in cache_dirs:
    if os.path.exists(cache_dir):
        shutil.rmtree(cache_dir)
        print(f"Removed cache directory: {cache_dir}")

Removed cache directory: /root/.cache/huggingface/datasets/ruslanmv___ai-medical-dataset/default/
Removed cache directory: /root/.cache/huggingface/datasets/


Save JSONL format dataset to empty list for training

In [ ]:
# Buka file JSONL
with jsonlines.open('ai_medical_dataset_train.jsonl', mode='r') as reader:
    # Membaca setiap baris dari file
    for features in reader:
        # Format teks instruksi dan tanggapan
        template = "Instruction:\n{Patient}\n\nResponse:\n{Doctor}"
        data.append(template.format(**features))

KeyError: 'Patient'